In [ ]:
# calc_SNR.ipynb
# Script tp get the SNR for an APF spectrum (or directory of spectra of single star)
# Last modified 8/12/20 by Anna Zuckerman 

from os import listdir
from os.path import isfile, join
import astropy.io.fits 
import numpy as np
import pandas as pd
import os

def get_SNR(path_name, file, apf_name_conv):
    # deal with case where path_name is a directory containing multiple spectra of the same star
    try:
        filenames = [f for f in listdir(path_name) if isfile(join(path_name, f))]
        mult_spect = True
    except NotADirectoryError: # path to one file
        mult_spect = False
    if not(mult_spect):
        filename = file.split('.')[0] + '.' + file.split('.')[1]
        hdul = astropy.io.fits.open(path_name) 
        data = hdul[0].data
        order_data = data[45,:]
        HIP_name = (apf_name_conv[apf_name_conv['FILENAME'] == filename])['HIP_NAME'].tolist()[0]
        SNR = np.sqrt(np.median(order_data))
    elif mult_spect:
        order_data = np.zeros([4608,1])
        for spect_file in filenames:
            hdul = astropy.io.fits.open(path_name + '/' + spect_file) 
            order_data = np.add(order_data,(hdul[0].data)[45])
        SNR = np.sqrt(np.median(order_data))
        HIP_name = file.split('_')[0]
    return SNR, HIP_name

apf_name_conv = pd.read_csv('apf_name_conversion_updated.csv')
filepath = './APF_spectra/apf_spectra' #input('Enter path to spectra files: ')   
filelist = sorted(os.listdir(filepath))
filelist.remove('.ipynb_checkpoints')
filelist.remove('HIP5643_spectra') # errors on this and not due to naming (GJ54.1)
all_SNR = []
filenames = []
HIP_names = []
for file in filelist: # sorted makes it go alphabetically
    print('Calculating SNR for ' + file)
    SNR, HIP_name = get_SNR(str(filepath+'/'+file), file, apf_name_conv)
    all_SNR = all_SNR + [SNR]
    HIP_names = HIP_names + [HIP_name]
SNR_results = pd.DataFrame(list(zip(HIP_names, all_SNR)), columns = ['HIP_NAME', 'SNR']) 
SNR_results.to_csv('SNR_results.csv')